<a href="https://colab.research.google.com/github/linjimzen/waymo_project/blob/main/TFrecord_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 導入lib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/aicar'

/content/drive/MyDrive/Colab Notebooks/aicar


In [ ]:
!pip3 install waymo-open-dataset-tf-2-12-0==1.6.4
!pip3 install Pillow==9.2.0

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
from waymo_open_dataset import dataset_pb2 as open_dataset

In [ ]:
'''
def get_path(types = 'train'):
    FILE_LIST = []
    # TODO : change path for yourself
    #DATA_PATH = './' + types
    DATA_PATH = types
    for record in os.listdir(DATA_PATH):
        if record.split('.')[0] == 'tfrecord':
            FILE_LIST.append(DATA_PATH + '/' + record)

    return FILE_LIST
'''

"\ndef get_path(types = 'train'):\n    FILE_LIST = []\n    # TODO : change path for yourself\n    #DATA_PATH = './' + types\n    DATA_PATH = types\n    for record in os.listdir(DATA_PATH):\n        if record.split('.')[0] == 'tfrecord':\n            FILE_LIST.append(DATA_PATH + '/' + record)\n\n    return FILE_LIST\n"

/content/drive/MyDrive/Colab Notebooks/aicar/tfrecord/train/training_segment-10017090168044687777_6380_000_6400_000_with_camera_labels.tfrecord

In [ ]:
path = './tfrecord/test_2'
for filename in os.listdir(path):
  print(filename)


training_segment-10584247114982259878_490_000_510_000_with_camera_labels.tfrecord


In [ ]:
# TODO : change path for yourself
RAWDATA_PATH = './tfrecord/test_2'
IMAGE_PATH = './dataset/test2/images/'
LABEL_PATH = './dataset/test2/labels/'
def preprocessing(RAWDATA_PATH):
    img_counter = 3000
    zero_filled = 6 # 命名時補0的數量上限
    round_limit = 4
    classes_count = [0,0,0] # 紀錄車子,行人,自行車騎士的數量
    mapping = { 1:0, 2:1, 4:2, 3:3, 5:4} # image name與frame.camera_labels順序之間的對應 (3 4 相反)
    for FILENAME in os.listdir(RAWDATA_PATH):
        print(f"====================================\n\n NOW is {FILENAME} \n\n====================================")
        dataset = tf.data.TFRecordDataset(RAWDATA_PATH+'/'+FILENAME, compression_type='')
        # 取出每一幀的數據
        for data in dataset:
            frame = open_dataset.Frame()
            frame.ParseFromString(bytearray(data.numpy()))
            # 將每個camera的image存成jpg檔 每幀共包含5個 camera
            for image in frame.images:
                # 沒有任何label則不儲存，直接下一個image

                if not frame.camera_labels[mapping[image.name]].labels:
                    continue

                # Part 1 :儲存圖片
                save_PATH = IMAGE_PATH + 'id_' + str(image.name) + 'number_' + str(img_counter).zfill(zero_filled) + '.jpg'
                # 從 bytes格式轉為 uint8格式，再解碼成圖片格式 BGR
                img = cv2.imdecode(np.frombuffer(image.image, np.uint8), cv2.IMREAD_COLOR)
                # (BGR -> RGB)
                cv2.imwrite(save_PATH, img)

                # Part 2 :儲存label
                save_PATH2 = LABEL_PATH + 'id_' + str(image.name) + 'number_' + str(img_counter).zfill(zero_filled) + '.txt'
                # camera的長寬 用來做 normalization
                camera_width = frame.context.camera_calibrations[image.name-1].width
                camera_height = frame.context.camera_calibrations[image.name-1].height
                # 取得每個Label的 information
                information = frame.camera_labels[mapping[image.name]].labels

                file = open(save_PATH2, 'w+')
                # 儲存每個object的bounding box資訊 (class_id, x_center, y_center, width, height), 需要Normalize至[0,1]
                for objects in information:
                    bbox = objects.box
                    class_id = objects.type
                    classes_count[objects.type//2] += 1 # vehicle:1, pedestrian:2, cyclist:4 # unknown:0 signs:3
                    x = round(bbox.center_x/camera_width, round_limit)
                    y = round(bbox.center_y/camera_height, round_limit)
                    # 這邊的length對應到的是 width , width對應到 height
                    width = round(bbox.length/camera_width, round_limit)
                    height = round(bbox.width/camera_height, round_limit)
                    file.write(f'{class_id} {x} {y} {width} {height}\n')
                file.close()
                img_counter += 1
    print(f'Vehicle : {classes_count[0]}')
    print(f'Pedestrian : {classes_count[1]}')
    print(f'Cyclist : {classes_count[2]}')
    print(f'img_counter : {img_counter-1}')

In [ ]:
preprocessing(RAWDATA_PATH)


 NOW is training_segment-10584247114982259878_490_000_510_000_with_camera_labels.tfrecord 

Vehicle : 4249
Pedestrian : 96
Cyclist : 0
img_counter : 3623


# 照片轉影片

In [ ]:
from pathlib import Path
import tqdm
import cv2

def set_camera(filename: str, width: int, height: int,
               fps: float = 30.0, fourcc: str = 'mp4v') -> cv2.VideoWriter:
    return cv2.VideoWriter(
        filename,
        fourcc=cv2.VideoWriter_fourcc(*fourcc),
        fps=fps,
        frameSize=(width, height)
    )

In [ ]:
batch_images = sorted(Path('/content/drive/MyDrive/Colab Notebooks/aicar/dataset/pic2mov').glob('*.jpg'))
video_output_path = 'video2.mp4'

camera = None
for ip in tqdm.tqdm(batch_images):
    frame = cv2.imread(ip.as_posix())

    # process ...

    if camera is None:
        h, w, _ = frame.shape
        camera = set_camera(video_output_path, w, h)
    camera.write(frame)
camera.release()

100%|██████████| 232/232 [00:17<00:00, 13.01it/s]
